In [1]:
# OceanRAIN reading, first attempt mid Nov 2017.
#  initially thought of running ARTS with OR PSD...


#import numpy as np
import h5py
#import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
#import sys
#sys.path.append('/home/dudavid/src/pyco/')
#from ddfunk import *
#import glob

In [2]:
%env ARTS_INCLUDE_PATH=/home/dudavid/arts/controlfiles/
%env ARTS_DATA_PATH=/home/dudavid/arts/arts-xml/
#%env ARTS_BUILD_PATH=/home/simonpf/build/arts/
%env ARTS_BUILD_PATH=/home/dudavid/arts/build/
#%load_ext autoreload
#%autoreload 2
%matplotlib inline

from typhon.arts.workspace import Workspace, arts_agenda
ws = Workspace()
ws.execute_controlfile("general/general.arts")
ws.execute_controlfile("general/continua.arts")
ws.execute_controlfile("general/agendas.arts")
ws.execute_controlfile("general/planet_earth.arts")

env: ARTS_INCLUDE_PATH=/home/dudavid/arts/controlfiles/
env: ARTS_DATA_PATH=/home/dudavid/arts/arts-xml/
env: ARTS_BUILD_PATH=/home/dudavid/arts/build/
Loading ARTS API from: /home/dudavid/arts/build/src/libarts_api.so
ARTS[94727632596752]: Executing /home/dudavid/arts/controlfiles/general/general.arts
ARTS[94727632596752]: {
ARTS[94727632596752]: - verbosityInit
ARTS[94727632596752]: - scat_speciesSet
ARTS[94727632596752]: - MatrixSet
ARTS[94727632596752]: - Tensor4SetConstant
ARTS[94727632596752]: - ArrayOfStringSet
ARTS[94727632596752]: - Touch
ARTS[94727632596752]: - FlagOff
ARTS[94727632596752]: - MatrixSet
ARTS[94727632596752]: - NumericSet
ARTS[94727632596752]: - ArrayOfStringSet
ARTS[94727632596752]: - Tensor3SetConstant
ARTS[94727632596752]: - Tensor3SetConstant
ARTS[94727632596752]: - Tensor3SetConstant
ARTS[94727632596752]: - Tensor3SetConstant
ARTS[94727632596752]: - Tensor3SetConstant
ARTS[94727632596752]: - Tensor3SetConstant
ARTS[94727632596752]: - IndexSet
ARTS[94727632

In [3]:
dire = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/'
fi = 'OceanRAIN__W__RV-SonneI_DFCG__UHAM-ICDC__v1_0.nc' #one of smaller W files
#files = glob.glob(dire+'*.nc')
#print(files)

In [4]:
f = h5py.File(dire+fi)

t  = np.array(f['air_temperature']) # in C
#c  = np.array(f['ceiling'])
jd = np.array(f['julian_date']) # days since jan 1 1994
la = np.array(f['latitude']) # deg N
lo = np.array(f['longitude']) # deg E
ss = np.array(f['sea_surface_temperature']) # in C
wst= np.array(f['true_wind_speed']) # m/s
w10 = np.array(f['wind_speed_in_10m_height']) # m/s
rrd= np.array(f['theoretical_rain_rate_disdrometer']) # mm/h
#rrg= np.array(f['rain_gauge_precipitation_rate']) # mm/h
#rro= np.array(f['ODM470_precipitation_rate_R']) # mm/h
pf = np.array(f['precip_flag']) # 1 (there are 2)
# 0=rain, 1=snow, 2=mixedphase, 3=noprecip(true-zero), 4/5=nodata, 9=misval
#dsd params:
mu = np.array(f['shape_parameter_of_normalized_gamma'])
dm = np.array(f['mass_weighted_mean_diameter_of_normalized_gamma'])
No = np.array(f['intercept_of_normalized_gamma']) 
# From Testud et al. (2004): N(D)=No*F(D/Dm) 
#  where F(x)=const*((4+mu)^(4+mu))/gamma(4+mu) * x^mu * exp(-(4+mu)x)
# can calculate LWC by integrating over N(D)*D^3*dD

#inf = info(ss,-99.9)
#inf = info(wst)
#inf = info(rrd,-999.0,'rr disd')
#inf = info(dm,-999.0)

In [6]:
# get some ARTS stuff set up with workspace methods

# define absorbing species and sensor (here using metmm library, used again below)
ws.Copy(ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA)
ws.abs_speciesSet(species=["H2O-PWR98","O2-PWR93","N2-SelfContStandardType"])
ws.ReadXML( ws.abs_lines, "instruments/metmm/abs_lines_metmm.xml.gz")
ws.abs_lines_per_speciesCreateFromLines()

ws.stokes_dim = 1 # 2?
ws.iy_unit = "PlanckBT" # work in tb space
ws.cloudboxOff()
#ws.cloudboxSetFullAtm()

In [ ]:
# define 'cloud box' where rain drops reside --start here next
#ws.scat_speciesSet(species=['MGD_LWC'])
#ws.cloudboxSetAutomatically()
#ws.cloudboxSetManually()
#ws.

In [7]:
ws.atmosphere_dim = 1 
p = np.array([1000.,975.,950.,925.,900.,850.,800.,750.,700.,650.,600.,\
              550.,500.,400.,300.,200.,100.])*100.0
ws.p_grid = 0.5 * (p[1:] + p[:-1])
ws.AtmRawRead( basename = "planets/Earth/Fascod/tropical/tropical") 
#tropical atmosphere assumed for now
ws.AtmFieldsCalc()
ws.AbsInputFromAtmFields()

ws.z_surface = np.asarray(ws.z_field)[0]
ws.t_surface = np.asarray(ws.t_field)[0]
alt = np.asarray(ws.z_field).ravel() * .001 # altitude in km (for plotting)


In [8]:
# sensor setup (channels, geometry)
ws.ArrayOfIndexCreate("channels")
ws.channels = [0,1,14] # channel index values given in metmm sensor description
ws.ArrayOfIndexCreate("viewing_angles")
ws.viewing_angles = [0] # index of stored viewing angles in metmm file
# 0 index viewing angle is 48.33deg (off-nadir) for amsua, 14 1.67deg (nadir)

ws.sensor_pos  = np.array([[850e3]]) # 850km
ws.sensor_time = np.array([0.0])
ws.sensor_los  = np.array([[180.0]]) # nadir viewing

ws.IndexCreate("met_mm_accuracy") 
ws.met_mm_accuracy = 1 # points within each bandwidth to comute abs characteristics

ws.execute_controlfile("instruments/metmm/sensor_descriptions/prepare_metmm.arts")
ws.execute_controlfile("instruments/metmm/sensor_descriptions/sensor_amsua.arts")
ws.execute_controlfile("instruments/metmm/sensor_descriptions/apply_metmm.arts")
ws.execute_controlfile("instruments/metmm/common_metmm.arts")

ARTS[94762409169696]: Executing /home/dudavid/arts/controlfiles/instruments/metmm/sensor_descriptions/prepare_metmm.arts
ARTS[94762409169696]: {
ARTS[94762409169696]: - ArrayOfArrayOfIndexCreate
ARTS[94762409169696]: - ArrayOfIndexCreate
ARTS[94762409169696]: - VectorCreate
ARTS[94762409169696]: - ArrayOfIndexCreate
ARTS[94762409169696]: - NumericCreate
ARTS[94762409169696]: - VectorCreate
ARTS[94762409169696]: - IndexCreate
ARTS[94762409169696]: }

ARTS[94762409169696]: Executing /home/dudavid/arts/controlfiles/instruments/metmm/sensor_descriptions/sensor_amsua.arts
ARTS[94762409169696]: {
ARTS[94762409169696]: - MatrixSet
ARTS[94762409169696]: - MatrixSet
ARTS[94762409169696]: - ArrayOfStringSet
ARTS[94762409169696]: - VectorSet
ARTS[94762409169696]: - Touch
ARTS[94762409169696]: - Delete
ARTS[94762409169696]: - ArrayOfIndexSet
ARTS[94762409169696]: - Append
ARTS[94762409169696]: - ArrayOfIndexSet
ARTS[94762409169696]: - Append
ARTS[94762409169696]: - ArrayOfIndexSet
ARTS[94762409169

In [9]:
ws.atmfields_checkedCalc( bad_partition_functions_ok = 1 )
ws.abs_xsec_agenda_checkedCalc()
ws.abs_lookupSetup()
ws.abs_lookupCalc()

ws.surface_scalar_reflectivity = np.array([0.5]) # nominal albedo for surface
ws.propmat_clearsky_agenda_checkedCalc()
ws.atmgeom_checkedCalc()
ws.cloudbox_checkedCalc()
ws.sensor_checkedCalc()
ws.jacobianOff()

In [10]:
#ws.DoitInit()
#ws.scatteringDoit(___)


ws.yCalc()
y = ws.y.value
print("Tb out: ", y)

Tb out:  [ 226.42440628  191.85874406  256.21720732]
